In [ ]:
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
from google.colab.patches import cv2_imshow
import matplotlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
mypath = '/content/drive/My Drive/fractal_test_images/multi/'

#Import and Process Images

In [ ]:
# Import and Process Images
filenames = sorted([f for f in listdir(mypath) if isfile(join(mypath, f))])
images = []

# Iterate through folder
for i in range(len(filenames)):
  image = cv2.imread(mypath+filenames[i], 0)
  # Add image to list
  images.append(image)
  print(f'{filenames[i]} - {np.shape(image)}') # Print filename and dimensions

# Box Counting Function

In [ ]:
def fractal_dimension(Z, threshold=0.5, thumb=False):

    # Only for 2d image
    assert(len(Z.shape) == 2)

    # From https://github.com/rougier/numpy-100 (#87)
    def boxcount(Z, k):
        S = np.add.reduceat(
            np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
                               np.arange(0, Z.shape[1], k), axis=1)

        # We count non-empty (0) and non-full boxes (k*k)
        return len(np.where((S > 0) & (S < k*k))[0])

    # Transform Z into a binary array
    Z = (Z < threshold)
    
    # Minimal dimension of image
    p = min(Z.shape)

    # Greatest power of 2 less than or equal to p
    n = 2**np.floor(np.log(p)/np.log(2))

    # Extract the exponent
    n = int(np.log(n)/np.log(2))

    # Build successive box sizes (from 2**n down to 2**1)
    sizes = 2**np.arange(n, 1, -1)

    # Actual box counting with decreasing size
    counts = []
    for size in sizes:
        counts.append(boxcount(Z, size))

    # Show image being analyzed 
    if thumb == True:
      matplotlib.pyplot.imshow(Z)

    # Fit the successive log(sizes) with log (counts)
    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
    return -coeffs[0]

##K-Means Image Segmentation 

In [ ]:
# K-Means function
def img_segment(img, k):
  dims = img.shape # Store 2d shape
  img = np.float32(img.reshape(-1,1)) # Reshape to 1d

  # K-Means steps
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
  passes=10
  label = cv2.kmeans(img, k, None, criteria, passes, cv2.KMEANS_PP_CENTERS)[1]
  labeled_img = label.reshape(dims)

  # Make mask for each segmented region
  regions = [] 
  for region in np.unique(label):
    mask = np.array([0 if x == region else 1 for x in label])
    mask = mask.reshape(dims)
    regions.append(mask)

  return regions

In [ ]:
# Run K-Means segmentation
regions = img_segment(images[1],2)

In [ ]:
len(regions)

##Crop Regions & Box Count

In [ ]:
for i in range(len(regions)):
  
  # Get region bounds and crop to them
  invert = -regions[i] + 1
  x, y = np.nonzero(invert)
  xl,xr = x.min(),x.max()
  yl,yr = y.min(),y.max()
  cropped = regions[i][xl:xr+1, yl:yr+1]

  # Box count cropped region
  D = fractal_dimension(-cropped + 1)

  # Plot regions and Fb results
  matplotlib.pyplot.figure()
  matplotlib.pyplot.title(f'{filenames[0]}: D~{D:.2f}')
  matplotlib.pyplot.imshow(-cropped)

# Uncropped Box Counting Results

In [ ]:
for i in range(len(regions)-1):
  
  # Box count region
  D = fractal_dimension(regions[i], thumb=True)
  
  # Plot regions and Fb results
  matplotlib.pyplot.title(f'{filenames[0]}: D~{D:.2f}')
  matplotlib.pyplot.figure()